In [52]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
cwd = os.getcwd()

In [8]:
bestiaryFiles = os.listdir("./bestiary")
spellFiles = os.listdir("./spells")

In [98]:
for creatureFile in bestiaryFiles:
    actions = []
    features = []
    legendaryAct = 0
    legendaryRes = 0
    mythic = False
    regional = False
    lair = False
    bonus = False
    reaction = False
    action = False
    with open(cwd + "\\bestiary\\" + creatureFile, "r") as f:
        soup = BeautifulSoup(f,'html.parser')
        name = soup.find_all("h1", {"class": "stats-name"})[0].string
        placesOfInterest = soup.find_all("td", {"class": "mon__sect-row-inner"})
        headersOfInterest = soup.find_all("h3", {"class":"mon__sect-header-inner"})
        for header in headersOfInterest:
            if(header.string == "Legendary Actions"):
                instances = soup(text=re.compile("take \d legendary actions"))
                match = re.search(r'take \d legendary actions', instances[0])
                legendaryAct = match.group()[5]
            elif(header.string == "Mythic Actions"):
                mythic = True
            elif(header.string == "Regional Effects"):
                regional = True
            elif(header.string == "Lair Actions"):
                lair = True
            elif(header.string == "Bonus Actions"):
                bonus = True
            elif(header.string == "Reactions"):
                reaction = True
            else:
                action = True
        currentIndexOfInterest = 0
        if(len(placesOfInterest)-len(headersOfInterest)>0):
            featureList = placesOfInterest[0]
            featureList = featureList.find_all("span",{'class': ["rd__h","rd__h--3"]})
            for feature in featureList:
                feature = feature.find("span",{'class':'entry-title-inner'}).string
                feature = re.sub("[.]", "", feature)
                feature = re.sub(" \(", "_(", feature)
                feature = feature.replace("\\","")
                featureSplit = feature.split("_")
                if(featureSplit[0]=="Legendary Resistance"):
                    legendaryRes = featureSplit[1][1]
                else:
                    features.append(featureSplit[0])
        if(action):
            pass
        if(bonus):
            pass
        if(reaction):
            pass
        if(legendaryAct!=0):
            pass
        if(mythic):
            print(name)
        if(lair):
            pass
        if(regional):
            pass
            
#         lengthNamedColumns = sum([mythic,lair,bonus,reaction,regional,legendaryAct!=0])
#         print(name, lengthNamedColumns)
#         for i, elements in enumerate(placesOfInterest):
#             elementsOfInterest = elements.find_all("div", {'class':['rd__b','rd__b--3']})
#             for element in elementsOfInterest:
#                 objectName = element.find("span",{'class':['entry-title-inner']})
#                 if(i == 0):
#                     features.append(objectName.string)
#                 else:
#                     actions.append(objectName.string)
#         totalFeatures = len(features)
#         print(name,legendaryAct,lair,regional,mythic)
#     if(legendaryAct > 0):
#         print(creatureFile,legendaryAct)
#         print(soup.prettify())

Ancient Dragon Turtle
Arasta
Aspect of Bahamut
Aspect of Tiamat
Black Greatwyrm
Blue Greatwyrm
Brass Greatwyrm
Bronze Greatwyrm
Copper Greatwyrm
Dullahan
Gold Greatwyrm
Green Greatwyrm
Hythonia
Red Greatwyrm
Silver Greatwyrm
Tromokratis
White Greatwyrm
